# Constrained Food Recommender

In this assignment, you will implement both Content Based and Collaborative Filtering Recommenders and backtracking search (or local search) on your own

100% finished homework should contain EDA, Item and User profiles generation, Content-Based Recommender, Collaborative Filtering Recommender, and soluton to CSP problem of assigning recommendations to brekfast, lunch and dinner.

In [15]:
import math
import numpy as np
import pandas as pd

from sklearn.preprocessing import normalize

## Data loading

You will work with subset of [Academic Yelp Dataset](https://www.kaggle.com/yelp-dataset/yelp-dataset) containing list of restaurants in **yelp_business.csv** and reviews of the users in **yelp_reviews.parquet**

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [78]:
# df_yelp_business = pd.read_csv("yelp_business.csv").drop(columns=["Unnamed: 0"])
# df_yelp_reviews = pd.read_parquet("yelp_reviews.parquet")
df_yelp_business = pd.read_csv("drive/MyDrive/yelp_business.csv").drop(columns=["Unnamed: 0"])
df_yelp_reviews = pd.read_parquet("drive/MyDrive/yelp_reviews.parquet")

# Leave only users with at least 3 reviews
users_count = df_yelp_reviews.groupby("user_id").count()[["business_id"]] 
users_to_use = users_count[users_count["business_id"] > 2]
df_yelp_reviews = df_yelp_reviews[df_yelp_reviews["user_id"].isin(users_to_use.index)]

## Exploratory data analysis

Here you will explore the data to find out what is the distribution of business categories, hours, places, user reviews, etc.

This step is needed to proceed later with item and user profiling and to clean your data if there are duplicates (e.g. duplicated reviews, the same businesses under different ids, categories tags which are highly correlated) or some artifacts not related to the main task.

(5 points)

In [79]:
df_yelp_business.head()

,address,attributes,business_id,categories,city,hours,is_open,latitude,longitude,name,postal_code,review_count,stars,state
0,404 E Green St,"{'RestaurantsAttire': ""u'casual'"", 'Restaurant...",pQeaRpvuhoEqudo3uymHIQ,"Ethnic Food, Food Trucks, Specialty Food, Impo...",Champaign,"{'Monday': '11:30-14:30', 'Tuesday': '11:30-14...",1,40.110446,-88.233073,The Empanadas House,61820,5,4.5,IL
1,4508 E Independence Blvd,"{'RestaurantsGoodForGroups': 'True', 'OutdoorS...",CsLQLiRoafpJPJSkNX2h5Q,"Food, Restaurants, Grocery, Middle Eastern",Charlotte,NaN,0,35.194894,-80.767442,Middle East Deli,28205,5,3.0,NC
2,300 John Street,"{'GoodForKids': 'True', 'RestaurantsTakeOut': ...",lu7vtrp_bE9PnxWfA8g4Pg,"Japanese, Fast Food, Food Court, Restaurants",Thornhill,NaN,1,43.820492,-79.398466,Banzai Sushi,L3T 5W4,7,4.5,ON
3,"4550 East Cactus Rd, #KSFC-4","{'GoodForKids': 'True', 'RestaurantsTakeOut': ...",vjTVxnsQEZ34XjYNS-XUpA,"Food, Pretzels, Bakeries, Fast Food, Restaurants",Phoenix,"{'Monday': '10:0-21:0', 'Tuesday': '10:0-21:0'...",1,33.602822,-111.983533,Wetzel's Pretzels,85032,10,4.0,AZ
4,9595 W Tropicana Ave,"{'Alcohol': ""u'none'"", 'WiFi': ""u'no'"", 'GoodF...",fnZrZlqW1Z8iWgTVDfv_MA,"Mexican, Restaurants, Fast Food",Las Vegas,NaN,0,36.099738,-115.301568,Carl's Jr,89147,15,2.5,NV


In [80]:
df_yelp_reviews.head()

,business_id,cool,date,funny,review_id,stars,text,useful,user_id
4,IS4cv902ykd8wj1TR0N3-A,0,2017-01-14 21:56:57,0,6TdNDKywdbjoTkizeMce8A,4,happy day finally canes near casa yes others g...,0,UgMW8bLE0QMJDCkQ1Ax5Mg
6,Pthe4qk5xh4n-ef-9bvMSg,0,2015-11-05 23:11:05,0,ZayJ1zWyWgY9S_TRLT_y9Q,5,really good place simple decor amazing food gr...,1,aq_ZxGHiri48TUXJlpRkCQ
9,Ws8V970-mQt2X9CwCuT5zw,1,2009-10-13 04:16:41,0,z4BCgTkfNtCu4XY5Lp97ww,4,twice nice laid back tried weekend southern me...,3,jOERvhmK6_lo_XGUBPws_w
16,d4qwVw4PcN-_2mK2o1Ro1g,0,2015-02-02 06:28:00,0,bVTjZgRNq8ToxzvtiVrqMA,1,10pm super bowl sunday already closed weak won...,0,2hRe26HSCAWbFRn5WChK-Q
22,9Jo1pu0y2zU6ktiwQm6gNA,20,2016-12-04 03:15:21,19,sgTnHfeaEvyOoWX4TCgkuQ,4,coconut fish cafe fantastic five stars fish ca...,24,A0j21z2Q1HGic7jW6e9h7A


In [74]:
# df_yelp_business = df_yelp_business[:500]
# df_yelp_reviews = df_yelp_reviews[:50]

In [81]:
# Removing dublicated businesses with same name, but different business_id

print(len(df_yelp_business))
df_yelp_business = df_yelp_business.drop_duplicates(subset=['business_id'])
print(len(df_yelp_business))

5748
5748


In [82]:
# Removing dublicated reviews with same user and business ids,
# but different review_id

print(len(df_yelp_reviews))
df_yelp_reviews.drop_duplicates(subset=['business_id'], keep='first', inplace=True)
print(len(df_yelp_reviews))

79934
5086


## Building recommender

First of all you should process user reviews to get the utility matrix containing ratings for users and businesses. There will be a lot of 0 in this matrix and it is better to store such matrices in the specialized data structure for sparce matrices. However, your working dataset is relatively small and we can use simple **pd.DataFrame** to proceed

In [83]:
def create_utility_matrix(reviews: pd.DataFrame, business: pd.DataFrame) -> pd.DataFrame:
    business_ids = business["business_id"].unique()
    users = reviews["user_id"].unique()
    ut_matrix = pd.DataFrame(0, columns=business_ids, index=users)
    for _, review in reviews.iterrows():
        ut_matrix.loc[review["user_id"], review["business_id"]] = review["stars"]
    
    # Rating normalization to (-1, 1) range (5 points)
    norm_ut_matrix = pd.DataFrame(normalize(ut_matrix, axis=1), columns=ut_matrix.columns)
    norm_ut_matrix.index = ut_matrix.index

    return norm_ut_matrix

df_utility_matrix = create_utility_matrix(df_yelp_reviews, df_yelp_business)
df_utility_matrix.head()

,pQeaRpvuhoEqudo3uymHIQ,CsLQLiRoafpJPJSkNX2h5Q,lu7vtrp_bE9PnxWfA8g4Pg,vjTVxnsQEZ34XjYNS-XUpA,fnZrZlqW1Z8iWgTVDfv_MA,rVBPQdeayMYht4Uv_FOLHg,fhNf_sg-XzZ3e7HEVGuOZg,LoRef3ChgZKbxUio-sHgQg,Ga2Bt7xfqoggTypWD5VpoQ,xFc50drSPxXkcLvX5ygqrg,tLpkSwdtqqoXwU0JAGnApw,Sd75ucXKoZUM2BEfBHFUOg,lK-wuiq8b1TuU7bfbQZgsg,LAoSegVNU4wx4GTA8reB6A,-qjn24n8HYF6It9GQrQntw,ZkzutF0P_u0C0yTulwaHkA,0QjROMVW9ACKjhSEfHqNCQ,RrapAhd8ZxCj-iue7fu9FA,7j0kor_fkeYhyEpXh4OpnQ,OWkS1FXNJbozn-qPg3LWxg,j9bWpCRwpDVfwVT_V85qeA,6GHwgKNlvfIMUpFaxgBjUA,Ir_QIzs-4o9ElOtiGuxJrw,0Y5Kzo8PWHTjk0tlfAKcDQ,JcsZvx-4yovFgCXOfd6KMg,MTx-Zdl_KcU_z9G832XAjg,a6d7UcYnRvbr4t-THg4pSQ,UFU8ONTkzEkcOk61XJ8JwQ,-C0AlwLuXpcP609madJZQQ,W7hCuNdn2gzehta6eSHzgQ,8nP8ghEpT6WFcM6tfqAaGA,8k62wYhDVq1-652YbJi5eg,8Hvp1tYKiQbBgGIwkCRK5g,39lLJK_rrYY2NYomSsQdUA,kQknjbOvtPmS3NVm-RhcdQ,0y6alZmSLnPzmG5_kP5Quw,3YjPlOX3VHzKguHetiR_3g,OGVHlFHSXjHuioOvm1wVqg,Q_dh08clYUPj13GmCRzIVA,34FYKG4pHNXbM9ZRRiJaGw,...,xnVkYE3iMp_aZniiCIuD0g,DmyS9b7ykIOo7XwYt5I9wg,Xs37o78aby0o-Wmvh5yYPg,jhj-r7aH3AlJyVmGtcHi-Q,Vjzg0VOQsBWw1TA3iLol-A,HfQm-rq78QfSeJmm0i1I9Q,r0byBoB7y_IH8uicEvyCqQ,dDqG-lKO9BRadoQ9fmWP-A,6-yG0OQe-mSRoz1R72MkKg,YO0fC7aJv8PzZ1MGf_F6Vg,nvJjfEPYFXj8sJZcbt0k-Q,BGGQOJQTQerEQu0kHbT_UQ,CvonRhKDJaH155xhtpz_iw,IZUDXIq5SULhQ5RGLCdB7g,VeFqptSzekFAc3FZOpi81Q,gVLzkqIAHIWro_ZxkpjbFg,GCen6oV-_6PfMP_uKN-dZw,P3dBcZh_Hmr1wVWFZn2b-g,2Wzet4CPV0glZYXqFGyqyg,XjrjVQfKpbcvOXda-5r1jQ,u1jADJ_yMcL8bRJre7hjMQ,VBCR0KKjFfzfcvV36KnkhA,02hhtAO83rZBU1hflleElA,6pKR-h3KN7AwgGOOYBbE2A,E50mr3xobsahb77IBRwVTQ,W1iwBUfcDxoOPfhwr6EOGQ,kw3DbQo6Pqo83FSDx8HQjQ,aGMU3qMFOQzG0DT2akMfng,1SJiW_mW6IlEe7hqMTnjYg,LkMtMHVetws5_7QfRjPtlg,2SfSzEd3B7WimeZac23zhg,1dV3qNEv8nNUAX1k3qdE2w,YHCseOJ93wJh0gBcii_2qA,TJt1W9haRm2DKuoZLQ69yA,wM8QNs7uSyDqMJKjBYFPCQ,gp_bu7Ah81qaBY3M0Leffw,PUKOr5bEI87TVHjwijT1xw,zV38gkkEeJ4cVRlSWWQTfQ,H1j34TgbrVZkxeww9xlJTw,F8M0IukXQqR50IRyocRQbg
UgMW8bLE0QMJDCkQ1Ax5Mg,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
aq_ZxGHiri48TUXJlpRkCQ,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
jOERvhmK6_lo_XGUBPws_w,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2hRe26HSCAWbFRn5WChK-Q,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A0j21z2Q1HGic7jW6e9h7A,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Content-Based Recommender

In [84]:
def build_business_profiles(business: pd.DataFrame) -> pd.DataFrame:
    # TODO: Feature engineering (10 points)

    # Getting all categories
    categories = []
    for _, info in business.iterrows():
        for category in info['categories'].split(', '):
            if category not in categories:
                categories.append(category)
    
    # Creating data frane for business profiles
    business_ids = business['business_id'].unique()
    business_profiles = pd.DataFrame(0, columns=categories, index=business_ids)

    # Place 1 for the categories that are present for current business
    for _, info in business.iterrows():
        for category in info['categories'].split(', '):
            business_profiles.loc[info['business_id'], category] = 1

    # Normalizing business_profiles
    # for business_id, categories in business_profiles.iterrows():
    #     categories_max = categories.max()
    #     categories_min = categories.min()
    #     # categories_mean = categories.mean()
    #     for (category_name, category_value) in categories.items():
    #         business_profiles.loc[business_id, category_name] = normalized(category_value, categories_min, categories_max)
    

    return business_profiles

df_business_profiles = build_business_profiles(df_yelp_business)
df_business_profiles.head()

,Ethnic Food,Food Trucks,Specialty Food,Imported Food,Argentine,Food,Restaurants,Empanadas,Grocery,Middle Eastern,Japanese,Fast Food,Food Court,Pretzels,Bakeries,Mexican,Burgers,American (Traditional),Chicken Wings,Lebanese,American (New),Hot Dogs,Chinese,Shopping Centers,Coffee & Tea,Cafes,Museums,Shopping,Local Flavor,Flowers & Gifts,Arts & Entertainment,Art Galleries,Florists,Egyptian,Pizza,Vietnamese,Buffets,Indian,Halal,Breakfast & Brunch,...,Beverage Store,Pancakes,Barbers,Town Car Service,Transportation,Septic Services,Limos,Historical Tours,Libraries,Churches,Amusement Parks,Restaurant Supplies,Wholesale Stores,Videos & Video Game Rental,Doctors,Cardiologists,Emergency Medicine,Pediatricians,Hospitals,Mongolian,Cosmetics & Beauty Supply,Furniture Stores,Home Decor,Dry Cleaning & Laundry,Accessories,Men's Clothing,Vinyl Records,Drugstores,South African,Escape Games,Hobby Shops,DJs,Fur Clothing,Pilates,Lighting Fixtures & Equipment,Ice Delivery,Furniture Repair,Party Equipment Rentals,Audio/Visual Equipment Rental,Furniture Rental
pQeaRpvuhoEqudo3uymHIQ,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
CsLQLiRoafpJPJSkNX2h5Q,0,0,0,0,0,1,1,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
lu7vtrp_bE9PnxWfA8g4Pg,0,0,0,0,0,0,1,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
vjTVxnsQEZ34XjYNS-XUpA,0,0,0,0,0,1,1,0,0,0,0,1,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
fnZrZlqW1Z8iWgTVDfv_MA,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [85]:
def build_user_profiles(utility_matrix: pd.DataFrame, 
                                   business_profiles: pd.DataFrame) -> pd.DataFrame:
    # Feature aggregation (5 points)

    user_profiles = pd.DataFrame(0, index=utility_matrix.index, columns=business_profiles.columns)
    
    # Iterate on all businesses for each user
    for user_id, businesses in utility_matrix.iterrows():
        # Finding the total number of restaurant categories currrent user visited
        # Example: 1 restaurant - 4 categories
        # So that, total number increases by 4
        all_businesses_means = []
        best_categories_max = -1 * math.inf
        # best_categories_min = math.inf

        for (business_id, _) in businesses.items():
            if business_id in business_profiles.index:
                curr_business = business_profiles.loc[[business_id]]

                categories_mean = float(curr_business.mean(axis=1))
                categories_max = float(curr_business.max(axis=1))
                if categories_max > best_categories_max:
                    best_categories_max = categories_max

                # categories_min = float(curr_business.min(axis=1))
                # if categories_min < best_categories_min:
                #     best_categories_min = categories_min
                
                # Counting the number of times current user visited
                # each restaurant category
                for category in curr_business:
                    user_profiles.loc[user_id, category] += int(curr_business[category])
                all_businesses_means.append(categories_mean)

        businesses_mean = sum(all_businesses_means) / len(all_businesses_means)

        # Normalizing
        for _, categories in user_profiles.loc[[user_id]].iterrows():
              for (category_name, category_value) in categories.items():
                  user_profiles.loc[user_id, category_name] = (category_value - businesses_mean) / best_categories_max

    return user_profiles

df_user_profiles = build_user_profiles(df_utility_matrix, df_business_profiles)
df_user_profiles.head()

KeyboardInterrupt: ignored

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity


def predict_content_ratings(user_profiles: pd.DataFrame, business_profiles: pd.DataFrame) -> pd.DataFrame:
    # TODO: Distance based rating prediction (5 points)
    predict_content = pd.DataFrame(0, index=user_profiles.index, columns=business_profiles.index)

    # user_count = 0
    for user_id, user_row in user_profiles.iterrows():
        cosine_result = cosine_similarity(business_profiles.append(user_row))
        prefered = (1 - cosine_result[-1])[:-1]
        # predict_content.loc[user_id] = prefered
        # print(f"For user {user_id} got {prefered}")

    # TODO: Pointwise/Pairwase training based prediction (optional for 10 extra points)
    
    return predict_content

df_content_predictions = predict_content_ratings(df_user_profiles, df_business_profiles)
df_content_predictions.head()

## Collaborative Filtering Recommender

In [ ]:
def predict_collaborative_ratings(utility_matrix: pd.DataFrame) -> pd.DataFrame:
    # TODO: User-item collaborative filtering based rating prediction (15 points)
    # TODO: UV-decomposition based rating prediction (optional for 10 extra points)
    pass

df_collaborative_predictions = predict_collaborative_ratings(df_utility_matrix)

## Evaluation

In [ ]:
def score_model(utility_matrix: pd.DataFrame, predicted_utility_matrix: pd.DataFrame, model_name="model_0"):
    # TODO: Implement these by hand (each metric 1 point)
    rmse_score = 0
    map_score = 0
    coverage_score = 0
    personalization_score = 0
    intra_list_similarity_score = 0
    
    print("{} RMSE {}".format(model_name, rmse_score))
    print("MAP: {}".format(model_name, map_score))
    print("Coverage: {}".format(model_name, coverage_score))
    print("Personalization: {}".format(model_name, personalization_score))
    print("Intra-list similarity: {}".format(model_name, intra_list_similarity_score))    

score_model(df_content_predictions, df_utility_matrix, "content-based approach")
score_model(df_collaborative_predictions, df_utility_matrix, "collaborative-filtering approach")

## Constraint Satisfaction Problem

We can work with the task of planing breakfast, lunch and dinner for particular user as Constraint Satisfaction Problem with

**Domain**: {all_businesses}

**Variables**: {breakfast, lunch, dinner}

**Constraints**: {constrainst regarding individual variable, or several variables at once}

We also have predicted ratings for every business and want to have personalized plan of restaurants. So we won't only satisfy our constraints, but also would like to get the maximum cumulative rating.

Take a look on prepared constraints and finish empty constraints in similar way (some of these constraints may require analytics on business data. e.g. to finish **has_coffee_constraint** you may need to determine all the categories which may include good coffee in their menu)

In [ ]:
def is_vegetarian_constraint(business_id):
    return "vegetarian" in df_yelp_business[df_yelp_business["business_id"] == business_id].categories.values[0].lower()

def has_coffee_constraint(business_id):
    # TODO: implement this constraint (1 point)
    return False

def has_alcohol_constraint(business_id):
    # TODO: implement this constraint (1 point)
    return False

def is_open_constraint(business_id):
    # TODO: implement this constraint (1 point)
    return False

def is_open_at_date_at_time_meta_constraint(weekday, time, business_id):
    # TODO: implement this constraint (1 point)
    return False

def is_open_at_monday_at_10am_constraint(business_id):
    return is_open_at_date_at_time("monday", "10:00", business_id)

def all_are_different_constraint(state):
    for time in ["breakfast", "dinner", "lunch"]:
        for _t in ["breakfast", "dinner", "lunch"]:
            if time == _t: continue
            business_categories = set(df_yelp_business[df_yelp_business["business_id"] == state[time]["business_id"]].categories.values[0].split(","))
            _business_categories = set(df_yelp_business[df_yelp_business["business_id"] == state[_t]["business_id"]].categories.values[0].split(","))
            if len(business_categories.intersection(_business_categories)) > \
                    len(business_categories.union(_business_categories)) // 2:
                return False
    return True

def all_are_in_the_same_city_constraint(state):
    # TODO: implement this constraint (1 point)
    return False

def all_are_in_the_same_region_meta_constraint(coordinates, threshold, state):
    # TODO: implement this constraint (1 point). Hint: use haversine distance https://pypi.org/project/haversine/
    return False

def all_are_in_test_region(state):
    return all_are_in_the_same_region_constraint({"lat": 40.110446, "lon": -115.301568}, 400, state)

def at_least_one_visited_place_constraint(state):
    # TODO: implement this constraint (2 points)
    # Make this constraint give more reward for more than one familiar place
    return False

def at_least_one_has_coffee_constraint(state):
    # TODO: implement this constraint (2 points)
    # Make this constraint give more reward for more than one place with coffee
    return False

In [ ]:
import random 

random.seed(42)
inspected_user = random.choice(df_yelp_reviews["user_id"].unique())

all_constraints = {
    "breakfast": [has_coffee_constraint, is_open_constraint, is_open_at_monday_at_10am_constraint],
    "lunch": [is_open_constraint],
    "dinner": [is_vegetarian_constraint, has_alcohol_constraint, is_open_constraint],
    "state": [at_least_one_has_coffee_constraint, at_least_one_visited_place_constraint, all_are_in_test_region,
             all_are_in_the_same_city_constraint, all_are_different_constraint]
}

def goal_test(state: dict, constraints: dict):
    cumulative_rating = state["breakfast"]["predicted_rating"]*state["lunch"]["predicted_rating"]*\
                        state["dinner"]["predicted_rating"]
    for k in constraints.keys():
        if k == "state":
            for c in constraints[k]:
                cumulative_rating *= c(state)
        else:
            for c in constraints[k]:
                cumulative_rating *= c(state[k]["business_id"])
    return cumulative_rating


def prepare_restaurants_plan(user_id: str, user_business_ratings: pd.DataFrame, constraints: dict):
    # TODO: assign breakfast, lunch and dinner by solving Constraint Satisfaction Problem 
    # maximizing total score and satisfying all the constraints (it should work with any configuration of constraints)
    
    # You can implement Backtracking (10) + Filtering (10) + Ordering (5) using goal_test
    # OR
    # Local Search (10) + Min-Conflicts heuristic (10) + Ordering (5) with modification of goal test to work as Min-Conflicts heuristic
    
    state = {"user_id" : user_id,
        "breakfast": 
                {"business_id": random.choice(user_business_ratings["business_id"].unique()),
                 "predicted_rating": 0},
            "lunch": 
    {"business_id": random.choice(user_business_ratings["business_id"].unique()),
                 "predicted_rating": 0},
            "dinner": {"business_id": random.choice(user_business_ratings["business_id"].unique()),
                 "predicted_rating": 0}}
    
    state_v = goal_test(state, constraints)

    
    return state

# TODO: replace df_utility_matrix with your best predictions
prepare_restaurants_plan(inspected_user, df_utility_matrix, all_constraints)